<a href="https://colab.research.google.com/github/sathsaraRasantha/Question_Answering_for_Sinhala_Language_using_Deep_NN_Architectures/blob/main/BiDAF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import re
from nltk.stem import WordNetLemmatizer
import numpy as np
import matplotlib.pyplot as plt
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer
from nltk import wordnet
from nltk.tokenize import word_tokenize
import string

import torch
import numpy as np
import pandas as pd
import pickle
import re, os, string, typing, gc, json
import spacy
from collections import Counter



stemmer = WordNetLemmatizer()

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Build a list of dictionaries with ( context , question , label ) as keys**

In [30]:
def parse_data(dataframe):
  QA_list=[]

  for i in range(0,len(dataframe['index'])):
    QA_dict={}
    QA_dict['id']=i
    QA_dict['context']=dataframe['context_translated'][i]
    QA_dict['question']=dataframe['question_translated'][i]
    QA_dict['label']=[dataframe['answer_start'][0],dataframe['answer_end'][i]]
    QA_list.append(QA_dict)
  return QA_list

# **Function to pre-process paragraph of text**

In [31]:

def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\u200d', '', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # define punctuation
        punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''


        preprocessed_text= ""
        for char in document:
          if char not in punctuations:
              preprocessed_text = preprocessed_text+ char
        
        return preprocessed_text

# **Function to create list of Context, Question and Context+Question lists**

In [32]:
def gather_text_for_vocab(df):
  context=[]
  for row1 in df['context_translated']:
    document1=row1
    text1=preprocess_text(document1)
    context.append(text1)

  
  question=[]
  for row2 in df['question_translated']:
    document2=row2
    text2=preprocess_text(document2)
    question.append(text2)

  answer=[]
  for row3 in df['text_translated']:
    document3=row3
    text3=preprocess_text(document3)
    answer.append(text3)

  lis=context+question
  return lis,context,question,answer

# **Function to build vocabulary of words , word to index and index to word mapping**

In [34]:
def build_word_vocab(lis):
   word_vocab=[]
   for sentences in lis:
      tokenized=sentences.split()
      word_vocab.append(tokenized)
   
   word2idx = {'__</e>__': 0, '__UNK__': 1} 
   idx2word={}
   for line in lis: 
     words=line.split()
     for word in words:
        if word not in word2idx: 
            word2idx[word] = len(word2idx)

   for item in word2idx:
     idx2word[word2idx[item]]=item


   return word2idx, idx2word, word_vocab


# **Function to create character vocabulary and character to index mapping**

In [35]:
def build_char_vocab(vocab_text):

    chars = []
    for sent in vocab_text:
        for ch in sent:
            chars.append(ch)

    char_counter = Counter(chars)
    char_vocab = sorted(char_counter, key=char_counter.get, reverse=True)
    print(f"raw-char-vocab: {len(char_vocab)}")
    high_freq_char = [char for char, count in char_counter.items() if count>=20]
    char_vocab = list(set(char_vocab).intersection(set(high_freq_char)))
    print(f"char-vocab-intersect: {len(char_vocab)}")
    char_vocab.insert(0,'<unk>')
    char_vocab.insert(1,'<pad>')
    char2idx = {char:idx for idx, char in enumerate(char_vocab)}
    print(f"char2idx-length: {len(char2idx)}")
    
    return char2idx, char_vocab

# **Function to create id's for pre-processed context words**

In [36]:
def context_to_ids(text, word2idx):
   
    context_tokens=text.split()
    # for paragraph in stripped:
    #     tokenized=word_tokenize(paragraph)
    #     context_tokens.append(tokenized)
    
    context_ids = [word2idx[word] for word in context_tokens]
    
    assert len(context_ids) == len(context_tokens)
    return context_ids

# **Function to create id's for pre-processed questions**

In [37]:
def question_to_ids(text, word2idx):
 

    question_tokens=text.split()
    question_ids = [word2idx[word] for word in question_tokens]
    
    assert len(question_ids) == len(question_tokens)
    return question_ids

In [9]:
from google.colab import files
uploaded = files.upload()

Saving partially_cleaned.csv to partially_cleaned.csv


In [39]:
from google.colab import files
uploaded = files.upload()

Saving english_sinhala_combined.csv to english_sinhala_combined.csv


In [40]:
data=pd.read_csv("english_sinhala_combined.csv")
data=data.drop(['Unnamed: 0',	'context',	'question',	'text'	],axis=1)
data

,context_translated,question_translated,text_translated
0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්
1,"2002 දී ප්‍රදර්ශනය ආරම්භ වූ දා සිට, අයිඩල් ජයග...",ඇමරිකානු අයිඩල් තරඟකරු ක්‍රිස් ඩොට්‍රි කුමන ප්...,උතුරු කැරොලිනාව
2,3 වන ශතවර්ෂයේ මැද භාගයේ සිට ජර්මානු ගෝත්‍රිකයන...,7 වන සියවසේ ඉදිකරන ලද දේ,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි
3,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි ...,90 දශකයේ දී කැනේ විසින් සාදන ලද බීට්ස් කවුද?,දේශීය කලාකරුවන්
4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්
...,...,...,...
19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ..."
19996,පැරණි ඉංග්‍රීසි භාෂාවෙන් ලතින් භාෂාවෙන් යම් නි...,ලතින් හෝඩිය හඳුන්වාදීමට පෙර පැරණි ඉංග්‍රීසි ලි...,රූනික් පද්ධතිය
19997,රූනික් පද්ධතිය... ඇරිස්ටෝෆනේස්ගේ නාට්‍ය ග්‍රීක...,ප්ලේටෝ සහ ඇරිස්ටෝටල් ලියුවේ කුමන ආකාරයේ සාහිත්...,දාර්ශනික පා
19998,ආරම්භක හා අවසන් දිනයන් දළ වශයෙන් දකුණු අර්ධගෝල...,දකුණු අර්ධගෝලයේ කුමන කාල පරිච්ඡේදයේදී චිලී ප්‍...,ගිම්හානය


In [11]:
df=pd.read_csv("partially_cleaned.csv")
df

,Unnamed: 0,index,context_translated,question_translated,text_translated,answer_start,answer_end
0,0,0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද,උතුරු අත්ලාන්තික්,0,17
1,1,4,ටුවාලුආන් භාෂාව සහ ඉංග්රීසි යනු ටුවාලු හි ජාති...,ඉංග්රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුමක්ද,ටුවාලුවාන්,56,66
2,2,8,ජේන් ඔස්ටන් ප්රධාන තදාසන්න ප්රදේශ අතර නිරිත දෙ...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්රදේශය කුමක්ද,මිඩ්ලෝතියන්,51,62
3,3,9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්ර පාක්ෂිකයන...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්රාන්ත...,හත,326,328
4,4,21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්රංශයේ අවනීතිය ස...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,157,171
...,...,...,...,...,...,...,...
9314,9314,19990,1900 වන විට මිනිසුන් 7531 ක් නගරයේ වාසය කළහ 19...,2006 දී ටක්සන්ගේ ජනගහනය කොපමණද,535000 කි,511,520
9315,9315,19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,313,318
9316,9316,19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,131,149
9317,9317,19995,දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට සාමාජිකත...,ලිබරල් ප්රජාතන්ත්රවාදයට රාජ්යයක් දක්වන සහයෝගය ...,පිහිටුවීමට සාමාජිකත්වය ප්රකාශ කිරීමට හෝ උද් ca...,27,101


In [12]:
msk = np.random.rand(len(df)) < 0.90661835
train_df = df[msk]
train_df=train_df.reset_index()
valid_df = df[~msk]
valid_df=valid_df.reset_index()

In [13]:
print(len(train_df))
print(len(valid_df))


8430
889


# **Data Preprocessing**

In [14]:
from torch import nn
import torch
import numpy as np
import pandas as pd
import pickle, time
import re, os, string, typing, gc, json
import torch.nn.functional as F
import spacy
from sklearn.model_selection import train_test_split
from collections import Counter
nlp = spacy.load('en')
%load_ext autoreload
%autoreload 2

In [41]:
lis,c,q,a = gather_text_for_vocab(data)

vocab_text=lis
print("Number of sentences in dataset: ", len(vocab_text))

Number of sentences in dataset:  40000


In [42]:
word2idx, idx2word, word_vocab = build_word_vocab(vocab_text)

In [43]:
len(word2idx)

85814

In [44]:
char2idx, char_vocab = build_char_vocab(vocab_text)

raw-char-vocab: 794
char-vocab-intersect: 191
char2idx-length: 193


In [48]:
q=0
context_ids_train=[]
for i in range(0,len(train_df['context_translated'])):
    try:                                                        
        ids = context_to_ids(train_df['context_translated'][i] , word2idx)
        context_ids_train.append(ids)
        
    except Exception as e:
        context_ids_train.append(e)
        q=q+1
        continue

In [49]:
q

4

In [50]:
r=0
question_ids_train=[]
for i in range(0,len(train_df['context_translated'])):
    try:                                                        
        ids = question_to_ids(train_df['context_translated'][i] , word2idx)
        question_ids_train.append(ids)
        
    except Exception as e:
        question_ids_train.append(e)
        r=r+1
        continue

In [51]:
r

4